<a href="https://colab.research.google.com/github/ricedavis/CSCE580-Fall2025/blob/main/ProjectB/steps_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Dataset

In [2]:
# import dataset
from google.colab import files
uploaded = files.upload()  # then select your CSV file


Saving IMDB Dataset.csv to IMDB Dataset.csv


In [3]:
import pandas as pd

df = pd.read_csv("IMDB Dataset.csv")
print(df.head())
print(df['sentiment'].value_counts())  # see how many positive/negative reviews


                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


# Preprocessing

In [4]:
import re

# minor text cleaning for DistilBERT
def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)  # remove HTML tags
    text = re.sub(r"[^a-z\s]", "", text)  # remove punctuation/numbers
    return text

df['clean_review'] = df['review'].apply(clean_text)


In [5]:
# convert sentiment to 0/1
df['label'] = df['sentiment'].map({'negative': 0, 'positive': 1})

For any supervised ML task, it's important to divide your dataset into seperate subsets for training, validation and testing. This ensures that the model learns patterns from one subset and is evaluated on unseen data.

In [9]:
from sklearn.model_selection import train_test_split

# using sklearn to split data

X_train, X_test, y_train, y_test = train_test_split(
    df['review'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, stratify=y_train, random_state=42
)

In [18]:
!pip install transformers

In [10]:
# tokenizing for DistilBERT

from transformers import DistilBertTokenizerFast
import tensorflow as tf

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

MAX_LENGTH = 128

def encode_texts(texts):
    return tokenizer.batch_encode_plus(
        texts.tolist(),
        max_length=MAX_LENGTH,
        padding='max_length',   # pad all sequences to same length
        truncation=True,
        return_attention_mask=True,
        return_tensors='tf'
    )

train_encodings = encode_texts(X_train)
valid_encodings = encode_texts(X_valid)
test_encodings = encode_texts(X_test)


# Fine-Tuning DistilBERT

In [16]:
!pip install torch transformers

In [17]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

# Tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Model for binary classification
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
